# 📝  Manual tuning (Exercise M3.01)

The goal is to write an exhaustive search to find the best parameters
combination maximizing the modelo generalization performance.

Here we use a small subset of the Adult Census dataset to make the code
faster to execute. Once your code works on the small subset, try to
change `train_size` to a larger value (e.g. 0.8 for 80% instead of
20%).

In [17]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("../datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.2, random_state=42)

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer(
    [('cat_preprocessor', categorical_preprocessor,
      selector(dtype_include=object))],
    remainder='passthrough', sparse_threshold=0)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

modelo = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42))
])


Use the previously defined modelo (called `modelo`) and using two nested `for`
loops, make a search of the best combinations of the `learning_rate` and
`max_leaf_nodes` parameters. In this regard, you will need to train and test
the modelo by setting the parameters. The evaluation of the modelo should be
performed using `cross_val_score` on the training set. We will use the
following parameters search:
- `learning_rate` for the values 0.01, 0.1, 1 and 10. This parameter controls
  the ability of a new tree to correct the error of the previous sequence of
  trees
- `max_leaf_nodes` for the values 3, 10, 30. This parameter controls the
  depth of each tree.

In [21]:
# Write your code here.
from sklearn.model_selection import cross_val_score

learning_rate = [0.01, 0.1, 1, 10]
max_leaf_nodes = [3, 10, 30]

best_score = 0
best_params = {}
for lr in learning_rate:
    for mln in max_leaf_nodes:
        print(f"Score basado en learning rate {lr:.3f}"
              f" y leaf node maximo {mln}... ", end="")
        modelo.set_params(
            classifier__learning_rate=lr,
            classifier__max_leaf_nodes=mln
        )
        scores = cross_val_score(modelo, data_train, target_train, cv=2)
        mean_score = scores.mean()
        print(f"score: {mean_score:.3f}")
        if mean_score > best_score:
            best_score = mean_score
            best_params = {'learning-rate': lr, 'leaf maximo por node': mln}
            print(f"Encontrar el mejor modeloo en base al score {best_score:.3f}!\n")

print(f"La mejor precisión obtenida es :{best_score:.3f}\n")
print(f"Los mejores parámetros encontrados son:\n {best_params}")

Score basado en learning rate 0.010 y leaf node maximo 3... score: 0.789
Encontrar el mejor modeloo en base al score 0.789!

Score basado en learning rate 0.010 y leaf node maximo 10... score: 0.813
Encontrar el mejor modeloo en base al score 0.813!

Score basado en learning rate 0.010 y leaf node maximo 30... score: 0.842
Encontrar el mejor modeloo en base al score 0.842!

Score basado en learning rate 0.100 y leaf node maximo 3... score: 0.847
Encontrar el mejor modeloo en base al score 0.847!

Score basado en learning rate 0.100 y leaf node maximo 10... score: 0.859
Encontrar el mejor modeloo en base al score 0.859!

Score basado en learning rate 0.100 y leaf node maximo 30... score: 0.857
Score basado en learning rate 1.000 y leaf node maximo 3... score: 0.855
Score basado en learning rate 1.000 y leaf node maximo 10... score: 0.835
Score basado en learning rate 1.000 y leaf node maximo 30... score: 0.828
Score basado en learning rate 10.000 y leaf node maximo 3... score: 0.288
Sco


Now use the test set to score the modelo using the best parameters
that we found using cross-validation in the training set.

In [20]:
# Write your code here.
best_lr = best_params['learning-rate']
best_mln = best_params['leaf maximo por node']

modelo.set_params(classifier__learning_rate=best_lr,
                 classifier__max_leaf_nodes=best_mln)
modelo.fit(data_train, target_train)
test_score = modelo.score(data_test, target_test)

print(f"Puntuación de la prueba después del ajuste de parámetros:{test_score:.3f}")

Puntuación de la prueba después del ajuste de parámetros:0.870
